**Helper functions and installing libraries**

In [1]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
API_key = 'JTFthLtGYuKFQK47MPvQcrabXJRJPk5nUEgaric4f17oP5VJcHyHqsqCMWXu'

#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    try:
        result = requests.get(api_call)
        result = result.json()
    except ValueError:
        print(f"Failed to decode JSON. Please check API call: {api_call}")

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False

    try:
        rate_limit = result['rate_limit']['remaining']
        reset = result['rate_limit']['resets_in_seconds']
    except:
        print('no rate limit available')

    return result, has_more, rate_limit, reset

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

def run_pages(api_first, api_second): 
    results, has_more, rate_limit, reset = run_api(f"{api_first}{api_second}")

    page = 2
    while has_more == True:
        if int(rate_limit) > 1:
            next_results, has_more, rate_limit, reset = run_api(f"{api_first}page={page}&{api_second}")
            results['data'] = results['data'] + next_results['data']
            page += 1
        else:
            print(f'Starting sleep timer for: {reset} seconds...')
            time.sleep(int(reset))
    else:
        pass
        
    return results

In [29]:
data = {'data':[]}
ids = [463,464] 

leagues = requests.get(f"https://api.sportmonks.com/v3/football/fixtures/19146812?api_token={API_key}&include=pressure")

leagues = leagues.json()

leagues

{'message': "You do not have access to the 'pressure' include",
 'link': 'https://docs.sportmonks.com/football/api/response-codes/other-exceptions',
 'code': 5002}

**Get league names**

In [51]:
leagues = requests.get(f"https://api.sportmonks.com/v3/football/leagues?api_token={API_key}")

leagues = leagues.json()

leagues

{'data': [{'id': 8,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Premier League',
   'active': True,
   'short_code': 'UK PL',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/8/8.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-27 16:30:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 9,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Championship',
   'active': True,
   'short_code': 'UK Champ',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/9/9.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-27 15:00:00',
   'category': 2,
   'has_jerseys': False},
  {'id': 24,
   'sport_id': 1,
   'country_id': 462,
   'name': 'FA Cup',
   'active': True,
   'short_code': 'UK FA Cup',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/24/24.png',
   'type': 'league',
   'sub_type': 'domestic_cup',
   'last_played_at': '2024-10-15 18:45:00',
   'category': 

In [58]:
wanted_leagues = ['Premier League', 'La Liga', 'Bundesliga', 'Serie A']
wanted_features = ['id', 'country_id', 'name', 'image_path']

leagues_df = {}

for league in leagues['data']:
    if league['name'] in wanted_leagues:
        for feature in wanted_features:
            app_dictionary(feature, league[f'{feature}'], leagues_df)
        
leagues_df = pd.DataFrame(leagues_df)
leagues_df = leagues_df.drop([3]).reset_index() #dropping russian Premier League
leagues_df.to_csv('data/leagues.csv')

**Get season ID by search**

In [6]:
season_names = ['2023', '2021', '2019', '2017', '2015', '2013']
season_data = {'data':[]}
leagues_df = pd.read_csv('data/leagues.csv')  
league_id = leagues_df['id'].to_numpy()

for season_name in season_names:
    first = 'https://api.sportmonks.com/v3/football/seasons/search/' + str(season_name) + '?'
    second = 'api_token=' + API_key
    results = run_pages(first, second)
    
    for season in results['data']:
        if season['league_id'] in league_id:
            season_data['data'].append(season)


In [73]:
seasonByLeagueID = pd.DataFrame(season_data['data'])
seasonByLeagueID.to_csv('data/seasons.csv')

In [86]:
seasonByLeagueID['league_id'].unique()

array([  8,  82, 564, 384])

Get venues by Season ID

In [90]:
venues_data = {'data':[]}
seasons_df = pd.read_csv('data/seasons.csv')  
seasons_ids = seasons_df['id'].to_numpy()

for season_id in seasons_ids:
    first = 'https://api.sportmonks.com/v3/football/venues/seasons/' + str(season_id) + '?'
    second = 'api_token=' + API_key 
    results = run_pages(first, second)
    
    try:
        for x in results['data']:
            x['season_id'] = str(season_id)
            venues_data['data'].append(x)
    except KeyError as e:
        print(f'KeyError at id: {season_id}')

Starting sleep timer for: 1567 seconds...
Starting sleep timer for: 1567 seconds...
Starting sleep timer for: 1567 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1566 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1565 seconds...
Starting sleep timer for: 1564 seconds...
Starting sleep timer for: 1564 seconds...
Starting sleep timer for: 1564 seconds...
Starting sleep timer for: 1564 seconds...
Starting sleep timer for: 1564 seconds...
Starting sleep timer for: 1564 seconds...
Starting sleep timer for: 1564 sec

In [91]:
venues_data = pd.DataFrame(venues_data['data'])
venues_data.to_csv('data/venues.csv')

In [93]:
print(len(venues_data['season_id'].unique()))
print(venues_data['country_id'].unique())

48
[ 462.  224. 1004.   11.   32.   nan  251.   44.  515. 3483.  353.]


Get Teams by Season ID

In [94]:
season_id = seasonByLeagueID['id'].to_numpy()
teams_data = {'data':[]}

for id in season_id:
    first = 'https://api.sportmonks.com/v3/football/teams/seasons/' + str(id) + '?'
    second = 'api_token=' + API_key
    print(f'Season id: {id}')
    results = run_pages(first, second)
    
    for team in results['data']:
        team['season_id'] = id
        teams_data['data'].append(team)

Season id: 19734
Starting sleep timer for: 3600 seconds...
Season id: 19744
Starting sleep timer for: 3600 seconds...
Season id: 19799
Starting sleep timer for: 3600 seconds...
Season id: 19806
Starting sleep timer for: 3600 seconds...
Season id: 21646
Starting sleep timer for: 3600 seconds...
Season id: 21694
Starting sleep timer for: 3600 seconds...
Season id: 21795
Starting sleep timer for: 3600 seconds...
Season id: 21818
Starting sleep timer for: 3599 seconds...
Season id: 17361
Starting sleep timer for: 3599 seconds...
Season id: 17420
Starting sleep timer for: 3599 seconds...
Season id: 17480
Starting sleep timer for: 3599 seconds...
Season id: 17488
Starting sleep timer for: 3599 seconds...
Season id: 18378
Starting sleep timer for: 3599 seconds...
Season id: 18444
Starting sleep timer for: 3599 seconds...
Season id: 18462
Starting sleep timer for: 3598 seconds...
Season id: 18576
Starting sleep timer for: 3598 seconds...
Season id: 12962
Starting sleep timer for: 3598 seconds.

In [95]:
teams_df = pd.DataFrame(teams_data['data'])

In [96]:
teams_df.to_csv('data/teams.csv')

In [111]:
#print(len(teams_df['season_id'].unique()))
#print(teams_df['season_id'].unique())
#np.array_equal(np.sort(teams_df['season_id'].unique()), np.sort(venues_data['season_id'].unique()))
print(np.sort(teams_df['season_id'].unique()))
print(np.sort([int(i) for i in venues_data['season_id'].unique()]))

[    3     7    10    12    13   215   216   217   218   219   802   853
  1581  1582  1583  1584  2059  2060  2061  2063  6397  8026  8442  8557
 12962 13005 13133 13158 16036 16264 16326 16415 17361 17420 17480 17488
 18378 18444 18462 18576 19734 19744 19799 19806 21646 21694 21795 21818]
[    3     7    10    12    13   215   216   217   218   219   802   853
  1581  1582  1583  1584  2059  2060  2061  2063  6397  8026  8442  8557
 12962 13005 13133 13158 16036 16264 16326 16415 17361 17420 17480 17488
 18378 18444 18462 18576 19734 19744 19799 19806 21646 21694 21795 21818]


**Get fixtures by team**

In [8]:
seasons_df = pd.read_csv('data/seasons.csv')  
start_date = seasons_df.sort_values(by=['starting_at'])['starting_at'].to_numpy()[0]
end_date = seasons_df.sort_values(by=['ending_at'], ascending=False)['ending_at'].to_numpy()[0]
#2012-07-20
#2024-06-02

Looping over periods of 100 days (limitation of the API)

In [7]:
# Initial date (starting point)
start_date = datetime(2012, 7, 20)  # Starting date in 'year-month-day'

# Number of intervals and days
days_interval = 100
total_days = 100

# Variable to store the previous date
previous_date = None

In [ ]:
# Loop over the 300 days in 100-day intervals\n",
for i in range(0, total_days, days_interval):
    current_date = start_date + timedelta(days=i)
    formatted_date = current_date.strftime('%Y-%m-%d')

    # Print the previous and current date,
    if previous_date:
        print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
        first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + str(previous_date) + '/' + str(formatted_date) + '?'
        second = 'api_token=' + API_key + '&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21;fixtureLeagues:8,82,384,486,564;fixtureSeasons=' 
        results = run_pages(first, second)
        try:
            for fixture in results['data']:
                fixture_data['data'].append(fixture)
        except:
            print(f'No data for team {id}')
    else: # Avoiding calling only the first date 
        print(f"First date: {formatted_date}") 

In [7]:
fixture_data = {'data': []}

first = 'https://api.sportmonks.com/v3/football/fixtures?'
second = 'api_token=' + API_key + '&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21' 
results = run_pages(first, second)

try:
    for fixture in results['data']:
        fixture_data['data'].append(fixture)
except:
    print(f'No data for team {id}')


In [4]:
season_data = pd.read_csv('data/seasons.csv')
str([i for i in season_data['id'].to_numpy()][:50]).strip('[ ]').replace(' ', '')

'19734,19744,19781,19799,19806,21646,21694,21713,21795,21818,17288,17361,17420,17480,17488,18375,18378,18444,18462,18576,12962,12996,13005,13133,13158,16021,16036,16264,16326,16415,13,219,802,828,853,6375,6397,8026,8442,8557,10,12,217,218,1583,1584,1916,1917,2061,2063'

In [5]:
fixture_data = {'data': []}
start_date = '2012-09-20'
end_date = '2012-12-20'

first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + start_date + '/' + end_date + '/' + '?'
second = 'api_token=' + API_key + '&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21;fixtureLeagues:8,82,384,486,564'
results = run_pages(first, second)

try:
    for fixture in results['data']:
        fixture_data['data'].append(fixture)
except:
    print(f'No data for team {id}')

In [9]:
play_sound()

In [8]:
fixture_data = pd.DataFrame(fixture_data['data'])
fixture_data.to_csv('data/fixture.csv')

In [8]:
fixture_data['league_id'].unique()

array([486,  82,   8, 564, 384], dtype=int64)

Saving as csv

In [98]:
fixture_data = pd.DataFrame(fixture_data['data'])

fixture_data.to_csv('data/fixture_data_full_test.csv')

***Get referees by Season ID***

In [47]:
referee_data = {'data':[]}
seasons_df = pd.read_csv('data/seasons.csv')  
season_id = seasons_df['id'].to_numpy()
print(season_id[0])

19734


In [48]:
for id in season_id:
    first = 'https://api.sportmonks.com/v3/football/referees/seasons/' + str(id) + '?'
    second = 'api_token=' + API_key + '&select=name,image_path'
    results = run_pages(first, second)

    try:
        for data in results['data']:
            referee_data['data'].append(data)
    except KeyError as e:
        print(f'KeyError at id: {id}')

Starting sleep timer for: 1482 seconds...
Starting sleep timer for: 1481 seconds...
Starting sleep timer for: 1480 seconds...
Starting sleep timer for: 1479 seconds...
Starting sleep timer for: 1478 seconds...
Starting sleep timer for: 1477 seconds...
Starting sleep timer for: 1477 seconds...
Starting sleep timer for: 1476 seconds...
Starting sleep timer for: 1475 seconds...
Starting sleep timer for: 1474 seconds...
Starting sleep timer for: 1473 seconds...
Starting sleep timer for: 1473 seconds...
Starting sleep timer for: 1472 seconds...
Starting sleep timer for: 1471 seconds...
Starting sleep timer for: 1470 seconds...
Starting sleep timer for: 1470 seconds...
Starting sleep timer for: 1469 seconds...
Starting sleep timer for: 1468 seconds...
Starting sleep timer for: 1468 seconds...
Starting sleep timer for: 1467 seconds...
Starting sleep timer for: 1466 seconds...
Starting sleep timer for: 1466 seconds...
Starting sleep timer for: 1465 seconds...
Starting sleep timer for: 1464 sec

In [49]:
referee_data

{'data': [{'name': 'David Coote',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/placeholder.png',
   'id': 11691,
   'country_id': 462,
   'sport_id': 1,
   'city_id': None},
  {'name': 'Geoff Eltringham',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/placeholder.png',
   'id': 11695,
   'country_id': 462,
   'sport_id': 1,
   'city_id': None},
  {'name': 'Mark Scholes',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/placeholder.png',
   'id': 12018,
   'country_id': 462,
   'sport_id': 1,
   'city_id': None},
  {'name': 'Matthew Wilkes',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/placeholder.png',
   'id': 12019,
   'country_id': 462,
   'sport_id': 1,
   'city_id': None},
  {'name': 'Marc Perry',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/placeholder.png',
   'id': 12025,
   'country_id': 462,
   'sport_id': 1,
   'city_id': None},
  {'name': 'Lee Betts',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/p

In [50]:
referee_data = pd.DataFrame(referee_data['data'])

referee_data.to_csv('data/referee_data.csv')

In [46]:
entity = requests.get(f"https://api.sportmonks.com/v3/football/referees?api_token={API_key}&select=name,image_path&include=refereestatistic")

entity = entity.json()

entity

{'message': "The requested include 'refereestatistic' does not exist",
 'link': 'https://docs.sportmonks.com/football/api/response-codes/include-exceptions',
 'code': 5001}

In [45]:
entity = requests.get(f"https://api.sportmonks.com/v3/my/filters/entity?api_token={API_key}")

entity = entity.json()

entity

{'data': {'aggregate': ['aggregateLeagues',
   'aggregateSeasons',
   'aggregateStages'],
  'coach': ['coachCountries', 'coachPlayers'],
  'coachstatistic': ['coachstatisticSeasons', 'currentseasons'],
  'coachstatisticdetail': ['coachstatisticdetailTypes'],
  'coachteam': [],
  'commentary': ['commentaryPlayers'],
  'event': ['eventTypes', 'eventPlayers'],
  'fixture': ['fixtureLeagues',
   'fixtureSeasons',
   'fixtureStates',
   'fixtureGroups',
   'fixtureStages',
   'participantsearch',
   'todaydate',
   'havingtrends',
   'venues',
   'deleted',
   'havingodds',
   'havingvaluebets',
   'havingexpectedgoals'],
  'fixtureballposition': [],
  'fixtureformation': [],
  'fixturereferee': ['fixturerefereeTypes'],
  'fixturesidelined': [],
  'fixturestatistic': ['fixturestatisticTypes'],
  'fixturetvstation': ['fixturetvstationCountries'],
  'group': [],
  'league': ['leagueLeagues', 'leagueCountries'],
  'lineup': ['lineupTypes', 'lineupPlayers'],
  'lineupdetail': ['lineupdetailType

**API call for events and add ons**

In [15]:
fixture_data['season_id'].unique()

array([    2,     3,     6,     7,     8,    10,    11,    12,    13,
          14,    23,    26,   176,   177,   178,   180,   189,   208,
         209,   210,   211,   212,   213,   214,   215,   216,   217,
         218,   219,   477,   478,   479,   480,   482,   484,   485,
         486,   487,   488,   556,   558,   560,   567,   649,   650,
         651,   652,   653,   654,   655,   656,   657,   658,   659,
         660,  1273,  1274,  1275,  1279,  1280,  1281,  1282,  1286,
         759,   765,   782,   804,   802,   801,   810,   816,   825,
         828,   848,   849,   852,   853,   858,   882,  1380,  1381,
        1382,  1383,  1384,  1385,  1386,  1387,  1388,  1389,  1390,
        1574,  1575,  1576,  1577,  1578,  1579,  1580,  1581,  1582,
        1583,  1584,  1586,  1588,  1587,  1590,  1591,  1592,  1595,
        1596,  1597,  1598,  1599,  1807,  1808,  1809,  1810,  1811,
        1812,  1813,  1814,  1815,  1816,  1817,  1829,  1830,  1831,
        1832,  1833,

In [10]:
weather_report = {}
referee_pr_fixture = {}
formation_pr_fixture = {}

#teams_data = {'data':[]}
for fixture in fixture_data['weather']:
        fixture['weatherreport'].append(weather_report)
        fixture['referees'][3].append(referee_pr_fixture)
        fixture['formations'][:].append(formation_pr_fixture)

KeyError: 'data'

In [101]:
pd.DataFrame.from_dict(weather_report)

In [84]:
extented_api_search = {'data':[]}

for id in fixture_data['id']:
    fixtures, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/{id}?api_token={API_key}&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")
    
    page = 2
    while has_more == True:
        next_features, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/{id}?page={page}&api_token={API_key}&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")
        fixtures['data'] = fixtures['data'] + next_features['data']
        page += 1
    
    try:
        extented_api_search['data'].append(fixtures['data'])
    except:
        print(f'Could not append for fixture: {id}')
    
    time.sleep(3)

{'message': "No result(s) found matching your request. Either the query did not return any results or you don't have access to it via your current subscription.", 'subscription': [{'meta': {'trial_ends_at': '2024-11-04 11:48:51', 'ends_at': '2024-11-21 10:53:06', 'current_timestamp': 1730069364}, 'plans': [{'plan': 'European Plan', 'sport': 'Football', 'category': 'Advanced'}], 'add_ons': [], 'widgets': []}], 'rate_limit': {'resets_in_seconds': 3600, 'remaining': 2999, 'requested_entity': 'Fixture'}, 'timezone': 'UTC'}
{'message': "No result(s) found matching your request. Either the query did not return any results or you don't have access to it via your current subscription.", 'subscription': [{'meta': {'trial_ends_at': '2024-11-04 11:48:51', 'ends_at': '2024-11-21 10:53:06', 'current_timestamp': 1730069364}, 'plans': [{'plan': 'European Plan', 'sport': 'Football', 'category': 'Advanced'}], 'add_ons': [], 'widgets': []}], 'rate_limit': {'resets_in_seconds': 3600, 'remaining': 2998, '

In [107]:
extented_api_search = {'data':[]}

fixtures, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/18546372?api_token={API_key}&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")

fixtures

{'data': {'id': 18546372,
  'sport_id': 1,
  'league_id': 384,
  'season_id': 19806,
  'stage_id': 77458057,
  'group_id': None,
  'aggregate_id': None,
  'round_id': 276020,
  'state_id': 5,
  'venue_id': 7305,
  'name': 'Spezia vs Juventus',
  'starting_at': '2023-02-19 17:00:00',
  'result_info': 'Juventus won after full-time.',
  'leg': '1/1',
  'details': None,
  'length': 90,
  'placeholder': False,
  'has_odds': True,
  'has_premium_odds': False,
  'starting_at_timestamp': 1676826000,
  'formations': [{'id': 395126,
    'fixture_id': 18546372,
    'participant_id': 625,
    'formation': '3-5-2',
    'location': 'away'},
   {'id': 395127,
    'fixture_id': 18546372,
    'participant_id': 345,
    'formation': '3-5-2',
    'location': 'home'}],
  'weatherreport': {'id': 168895,
   'fixture_id': 18546372,
   'venue_id': 7305,
   'temperature': {'day': 11.77,
    'morning': 10.58,
    'evening': 11.5,
    'night': 11.13},
   'feels_like': {'day': 11.51,
    'morning': 10.28,
    'ev

In [105]:
try:
    extented_api_search['data'].append(fixtures['data'])
except:
    print(f'Could not append for fixture: {id}')

In [106]:
extented_api_search

{'data': [{'id': 18546355,
   'sport_id': 1,
   'league_id': 384,
   'season_id': 19806,
   'stage_id': 77458057,
   'group_id': None,
   'aggregate_id': None,
   'round_id': 276019,
   'state_id': 5,
   'venue_id': 7220,
   'name': 'Empoli vs Spezia',
   'starting_at': '2023-02-11 14:00:00',
   'result_info': 'Game ended in draw.',
   'leg': '1/1',
   'details': None,
   'length': 90,
   'placeholder': False,
   'has_odds': True,
   'has_premium_odds': False,
   'starting_at_timestamp': 1676124000,
   'formations': [{'id': 392574,
     'fixture_id': 18546355,
     'participant_id': 345,
     'formation': '3-5-2',
     'location': 'away'},
    {'id': 392575,
     'fixture_id': 18546355,
     'participant_id': 397,
     'formation': '4-3-1-2',
     'location': 'home'}],
   'weatherreport': {'id': 164121,
    'fixture_id': 18546355,
    'venue_id': 7220,
    'temperature': {'day': 8.46,
     'morning': 0.86,
     'evening': 6.67,
     'night': 3.65},
    'feels_like': {'day': 8.46,
     

In [70]:
result = result.json()

In [71]:
result

{'message': "No result(s) found matching your request. Either the query did not return any results or you don't have access to it via your current subscription.",
 'subscription': [{'meta': {'trial_ends_at': '2024-11-04 11:48:51',
    'ends_at': '2024-11-21 10:53:06',
    'current_timestamp': 1729590039},
   'plans': [{'plan': 'European Plan',
     'sport': 'Football',
     'category': 'Advanced'}],
   'add_ons': [],
   'widgets': []}],
 'rate_limit': {'resets_in_seconds': 1106,
  'remaining': 47,
  'requested_entity': 'Fixture'},
 'timezone': 'UTC'}

In [51]:
play_sound()

In [52]:
weather_report = []
referee_pr_fixture = []
formation_pr_fixture = []

#teams_data = {'data':[]}
for fixture in extented_api_search['data']:
        weather_report.append(fixture['weatherreport'])
        formation_pr_fixture.append( fixture['formations'][:])

In [14]:
str([i for i in season_data['id'].to_numpy()][:5]).strip('[ ]').replace(' ', '')

'19734,19744,19781,19799,19806'

In [36]:
events

{'data': [{'id': 18535054,
   'sport_id': 1,
   'league_id': 8,
   'season_id': 19734,
   'stage_id': 77457864,
   'group_id': None,
   'aggregate_id': None,
   'round_id': 274669,
   'state_id': 5,
   'venue_id': 480,
   'name': 'Brighton & Hove Albion vs Newcastle United',
   'starting_at': '2022-08-13 14:00:00',
   'result_info': 'Game ended in draw.',
   'leg': '1/1',
   'details': None,
   'length': 90,
   'placeholder': False,
   'has_odds': True,
   'has_premium_odds': False,
   'starting_at_timestamp': 1660399200,
   'formations': [{'id': 336854,
     'fixture_id': 18535054,
     'participant_id': 20,
     'formation': '4-3-3',
     'location': 'away'},
    {'id': 336853,
     'fixture_id': 18535054,
     'participant_id': 78,
     'formation': '3-4-2-1',
     'location': 'home'}],
   'weatherreport': {'id': 104269,
    'fixture_id': 18535054,
    'venue_id': 480,
    'temperature': {'day': 29.75,
     'morning': 18.9,
     'evening': 27.83,
     'night': 20.56},
    'feels_lik

In [31]:
seasons = [i for i in season_data['id'].to_numpy()]
for index, row in fixture_data.iterrows():
    if row['season_id'] not in seasons:
        fixture_data.drop([index])

In [35]:
fixture_data

,id,sport_id,league_id,season_id,stage_id,group_id,aggregate_id,round_id,state_id,venue_id,...,details,length,placeholder,has_odds,has_premium_odds,starting_at_timestamp,formations,weatherreport,referees,events
0,463,1,8,2,2,NaN,NaN,43.0,5,209.0,...,None,90,False,False,False,1281786300,"[{'id': 2795920, 'fixture_id': 463, 'participa...",None,"[{'id': 292213, 'fixture_id': 463, 'referee_id...","[{'id': 75541358, 'type_id': 19, 'sub_type_id'..."
1,464,1,8,3,3,NaN,NaN,44.0,5,230.0,...,None,90,False,False,False,1376739900,"[{'id': 2796556, 'fixture_id': 464, 'participa...",None,"[{'id': 1568755, 'fixture_id': 464, 'referee_i...","[{'id': 30525448, 'type_id': 19, 'sub_type_id'..."
2,465,1,8,2,2,NaN,NaN,43.0,5,NaN,...,None,90,False,False,False,1281794400,"[{'id': 2795799, 'fixture_id': 465, 'participa...",None,"[{'id': 1986226, 'fixture_id': 465, 'referee_i...","[{'id': 30525641, 'type_id': 19, 'sub_type_id'..."
3,466,1,8,3,3,NaN,NaN,44.0,5,204.0,...,None,90,False,False,False,1376748000,"[{'id': 2796466, 'fixture_id': 466, 'participa...",None,"[{'id': 467137, 'fixture_id': 466, 'referee_id...","[{'id': 30525734, 'type_id': 19, 'sub_type_id'..."
4,467,1,8,2,2,NaN,NaN,43.0,5,339721.0,...,None,90,False,False,False,1281794400,"[{'id': 2795845, 'fixture_id': 467, 'participa...",None,"[{'id': 217177, 'fixture_id': 467, 'referee_id...","[{'id': 30525554, 'type_id': 19, 'sub_type_id'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154648,19321921,1,24,23787,77471893,NaN,NaN,NaN,1,982.0,...,None,90,False,True,True,1730559600,[],None,[],[]
154649,19321922,1,24,23787,77471893,NaN,NaN,NaN,1,339735.0,...,None,90,False,True,True,1730559600,[],None,[],[]
154650,19321923,1,24,23787,77471893,NaN,NaN,NaN,1,261.0,...,None,90,False,True,True,1730559600,[],None,[],[]
154651,19321924,1,24,23787,77471893,NaN,NaN,NaN,1,983.0,...,None,90,False,True,True,1730637000,[],None,[],[]
